In [8]:
import sys
print(sys.executable)

import numpy as np
from scipy.stats import rankdata, ttest_rel, ttest_1samp
from scipy.stats import zscore

from matplotlib import pyplot as plt
from scipy.stats import linregress
import pickle
import seaborn as sns

import nibabel as nib
subjects =  ['sub-EZ9NM','sub-TV7EF','sub-WLC4O','sub-3L109','sub-3UYSS','sub-JVPPA','sub-EDOP8',
             'sub-TIJD1','sub-AYDLR','sub-98UT7','sub-4G9A5','sub-YJBI8','sub-FG8PD','sub-BBN4K',
             'sub-F61IR','sub-M1ZRL','sub-NZJZN','sub-XZ9SS','sub-L05R3','sub-ZOVPY','sub-6PEOF',
             'sub-G6RPG','sub-TPNCU','sub-9ADGJ','sub-27IXQ','sub-NDX5S','sub-OE7EZ','sub-28E84',
             'sub-ENN9N','sub-28OBV','sub-8WJAP','sub-SPHBN','sub-WP8SX','sub-BG4CW','sub-AVQEZ',
             'sub-YLWD1','sub-0A4MV','sub-8PIML','sub-4T7NM','sub-X5RE0','sub-RUQON','sub-JVT46',
             'sub-RTFX9','sub-OWWAF','sub-IC2KG','sub-S3I4J','sub-XFQA1','sub-UKF5L','sub-Z0MJJ',
             'sub-9HMVR','sub-DBQ8H','sub-NAD3W','sub-KH33B','sub-JASQ6','sub-T6FHZ','sub-PQ8JD',
             'sub-NOVID','sub-XAKBX','sub-JVSIQ','sub-D1FKN','sub-R3JA3','sub-GPMCX','sub-H8QL5',
             'sub-6HW3V','sub-WF939']

fmri_data = []
for i, fname in enumerate(subjects):
    print(fname)
    path = f'/DATA/FilmMemory_preprocess/mrbean_scrambled/{fname}_scrambled_smoothing_scale.nii.gz'
    data = zscore(nib.load(path).get_fdata(), axis=-1)
    data = np.nan_to_num(data)
    fmri_data.append(data)

/home/jwpark/anaconda3/envs/NarrativePuzzle/bin/python3.10
sub-001
sub-002
sub-003
sub-004
sub-005
sub-006
sub-013
sub-014
sub-015
sub-016
sub-017
sub-018
sub-019
sub-020
sub-021
sub-022
sub-023
sub-024
sub-025
sub-026
sub-027
sub-028
sub-029
sub-030
sub-031
sub-032
sub-033
sub-034
sub-035
sub-038
sub-039
sub-040
sub-042
sub-043
sub-044
sub-049
sub-050
sub-057
sub-058
sub-059
sub-060
sub-061
sub-062
sub-063
sub-064
sub-072
sub-075
sub-078
sub-082
sub-083
sub-086
sub-087
sub-089
sub-090
sub-092
sub-095
sub-096
sub-099
sub-101
sub-103
sub-105
sub-107
sub-109
sub-111
sub-113


In [2]:
from multiprocessing import Pool
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
import time

content_score = np.load("../Figure_3/data/content_score.npy")
ordering_score = np.load("../Figure_3/data/ordering_score.npy")

with open('../utils/movie/backward_retrieval_related.pkl', 'rb') as f:
    backward_retrieval_related = pickle.load(f)
with open('../utils/movie/backward_retrieval_unrelated.pkl', 'rb') as f:
    backward_retrieval_unrelated = pickle.load(f)

with open('../utils/movie/forward_retrieval_related.pkl', 'rb') as f:
    forward_retrieval_related = pickle.load(f)
with open('../utils/movie/forward_retrieval_unrelated.pkl', 'rb') as f:
    forward_retrieval_unrelated = pickle.load(f)

def get_subject_similarity(i):
    x = voxel_indices[0][i]
    y = voxel_indices[1][i]
    z = voxel_indices[2][i]
    if np.sum(mni_mask[x-r:x+r+1, y-r:y+r+1, z-r:z+r+1]) > int(((2*r+1)**3)*0.50):
        sphere = sub_data[x-r:x+r+1, y-r:y+r+1, z-r:z+r+1, :].reshape(((2*r+1)**3,610)).T  #Time, Vox
        sphere = sphere[:, np.any(sphere, axis=0)]

        similarity = []
        for p, pair in enumerate(backward_retrieval_related):
            prev_patterns = sphere[pair[0]+wholebrain_backward_retrieval_offset-wholebrain_backward_retrieval_duration//2:pair[0]+wholebrain_backward_retrieval_offset-wholebrain_backward_retrieval_duration//2+wholebrain_backward_retrieval_duration, :].mean(axis=0)
            next_patterns = sphere[pair[1]+offset-duration//2:pair[1]+offset-duration//2+duration, :].mean(axis=0)
            similarity.append(1-cosine(prev_patterns, next_patterns))
        return np.nanmean(similarity)
    else:
        return 0

wholebrain_backward_retrieval_offset = int(np.mean([13, 15, 15, 15]))
wholebrain_backward_retrieval_duration = int(np.mean([9, 9, 9, 13]))
offset = -8
duration = 6
r = 3
mni_mask = nib.load('/DATA/FilmMemory_preprocess/supplement/mni_mask.nii').get_fdata()
n_voxels = np.sum(mni_mask)
print('Total number of voxels:', np.sum(mni_mask))
voxel_indices = np.where(mni_mask==1)


Total number of voxels: 69831.0


In [3]:
time_sta = time.time()
voxel_similarities = np.zeros((len(subjects), int(n_voxels)))
for s, sub in enumerate(subjects):
    sub_data = fmri_data[s]
    with Pool() as pool:
        voxel_similarity = pool.map(get_subject_similarity, [i for i in range(int(n_voxels))])
    voxel_similarities[s,:] = voxel_similarity
    time_end = time.time()
    print(f'Progress:{sub}, ETA:{(time_end-time_sta):3.3f}s')
    time_sta = time_end

Progress:sub-001, ETA:24.589s
Progress:sub-002, ETA:24.616s
Progress:sub-003, ETA:23.391s
Progress:sub-004, ETA:20.667s
Progress:sub-005, ETA:24.394s
Progress:sub-006, ETA:24.823s
Progress:sub-013, ETA:24.806s
Progress:sub-014, ETA:24.269s
Progress:sub-015, ETA:24.814s
Progress:sub-016, ETA:24.794s
Progress:sub-017, ETA:24.720s
Progress:sub-018, ETA:24.676s
Progress:sub-019, ETA:24.864s
Progress:sub-020, ETA:20.288s
Progress:sub-021, ETA:21.544s
Progress:sub-022, ETA:27.203s
Progress:sub-023, ETA:21.888s
Progress:sub-024, ETA:21.430s
Progress:sub-025, ETA:21.285s
Progress:sub-026, ETA:21.595s
Progress:sub-027, ETA:21.667s
Progress:sub-028, ETA:20.149s
Progress:sub-029, ETA:21.203s
Progress:sub-030, ETA:21.602s
Progress:sub-031, ETA:21.610s
Progress:sub-032, ETA:21.643s
Progress:sub-033, ETA:21.658s
Progress:sub-034, ETA:21.660s
Progress:sub-035, ETA:21.571s
Progress:sub-038, ETA:21.604s
Progress:sub-039, ETA:21.801s
Progress:sub-040, ETA:21.503s
Progress:sub-042, ETA:21.504s
Progress:s

In [5]:
backward_retrieval_corr_r= np.zeros(mni_mask.shape)
backward_retrieval_corr_p= np.ones(mni_mask.shape)
for i in range(int(n_voxels)):
    x = voxel_indices[0][i]
    y = voxel_indices[1][i]
    z = voxel_indices[2][i]
    #if np.sum(mni_mask[x-r:x+r+1, y-r:y+r+1, z-r:z+r+1]) > int(((2*r+1)**3)*0.50):
    corr_r, corr_p = pearsonr(voxel_similarities[:,i], ordering_score)
    backward_retrieval_corr_r[x,y,z] = corr_r
    backward_retrieval_corr_p[x,y,z] = corr_p

np.save('data/wholebrain_searchlight_backward_retrieval_corr_r.npy', backward_retrieval_corr_r)
np.save('data/wholebrain_searchlight_backward_retrieval_corr_p.npy', backward_retrieval_corr_p)

/home/jwpark/anaconda3/envs/NarrativePuzzle/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [1]:
import numpy as np
import nibabel as nib
backward_retrieval_corr_r = np.load('data/wholebrain_searchlight_backward_retrieval_corr_r.npy')
mni_mask = nib.load('/DATA/FilmMemory_preprocess/supplement/mni_mask.nii').get_fdata()
new_img = nib.Nifti1Image(backward_retrieval_corr_r, mni_mask.affine, mni_mask.header)
nib.save(new_img, 'data/wholebrain_searchlight.nii.gz')

AttributeError: 'memmap' object has no attribute 'affine'